## Anomaly detection class

In [19]:
class Anomaly_:
    def __init__(self, data, var , bootstrap=False, contamination=0.01, max_features=1.0,
        max_samples='auto', n_estimators=300, n_jobs=1, random_state=2018,
        verbose=0, var_target="Number"):
        self.data =  data
        self.var = var
        self.boostrap = bootstrap
        self.contamination = contamination
        self.max_features = max_features
        self.max_samples = max_samples
        self.n_estimators = n_estimators
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.verbose = verbose
        self.Model_ = None
        self.var_target = var_target
        self.Counter_= 0
        
    def __Inspection__(self):
        try:
            self.var_int64 = self.data.columns[np.where(self.data.dtypes.values=='int64')].tolist()
            self.var_float64 = self.data.columns[np.where(self.data.dtypes.values=='float64')].tolist()
            self.var_object = self.data.columns[np.where(self.data.dtypes.values=='O')].tolist()
            self.var_other_type = self.data.columns[np.where([i not in self.data.dtypes.values for i in ["int64","float64","O"]])].tolist()
        except:
            self.var_int64 = None
            self.var_float64 = None
            self.var_object = None
            self.var_other_type = None
            
    def Missing_values(self,NA_treatment="mean"):
        try: 
            self.__Inspection__()
            self.data[self.var_object]=self.data[self.var_object].fillna("__MISSING_VALUE__")
            self.data[self.var_other_type]=self.data[self.var_other_type].fillna("__MISSING_VALUE__")

            if NA_treatment == "mean":
                self.data[self.var_int64]=self.data[self.var_int64].fillna(self.data[self.var_int64].mean())
                self.data[self.var_float64]=self.data[self.var_float64].fillna(self.data[self.var_int64].mean())

            elif NA_treatment == "median":
                self.data[self.var_int64]=self.data[self.var_int64].fillna(self.data[self.var_int64].median())
                self.data[self.var_float64]=self.data[self.var_float64].fillna(self.data[
                    self.var_int64].median())
            else : 
                self.data[self.var_int64]=self.data[self.var_int64].fillna(0)
                self.data[self.var_float64]=self.data[self.var_float64].fillna(0)
        except:
            print("N/O")
            
    def __Preparation__(self):
        self.Missing_values()
        self.data_prep = pd.get_dummies(self.data[self.var])
        
    def __detection__(self):
        self.__Preparation__()
        self.Model_ =  IsolationForest(random_state=self.random_state, contamination=self.contamination, n_estimators = self.n_estimators)
        self.Model_.fit(self.data_prep)
        self.data["Abnormality_Score"]=self.Model_.decision_function(self.data_prep)
        self.data["Abnormality"]=self.Model_.predict(self.data_prep)
        self.Counter_ +=1
        
    def __Graph__(self, var_target =None , quantile=0.99, figsize=(20,10), label=None, title_=''):
        if var_target == None:
            var_target = self.var_target
        if self.Counter_ == 0 :
            self.__detection__()
        try :
            plt.figure(figsize=figsize)
            plt.plot(self.data.index, self.data[var_target], label=label)
            plt.scatter(self.data.index[self.data.Abnormality_Score<=(self.data.Abnormality_Score.quantile(1-quantile))],self.data[var_target][self.data.Abnormality_Score<=(self.data.Abnormality_Score.quantile(1-quantile))], c="red", label="Outliers")
            plt.axhline(self.data[var_target].mean(), c='red',linestyle=":",label="Average")
            plt.legend(loc=1)
            plt.title(title_)
            plt.show()
        except:
            print("Graph error")

## Display function

In [ ]:
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

## Analysis function

In [ ]:
def Analysis(Data, title="user", displayed = False):
    data = Data.reset_index()
    
    #Number of connection per days/date
    Data__ = data.groupby(["dow",
                           "date_id"]).count().reset_index().sort_values(by = ["dow",
                                                                                     "date_id"])[["dow",
                                                                                                  "date_id","timestamp"]].rename(columns={"timestamp":"Number"}).set_index(["dow","date_id"])
    #Number of connections per days/hours
    Data_H__ = data.groupby(["dow","date_id",
                      "Hour"]).count().reset_index().sort_values(by = [
    "dow","date_id","Hour"])[[
    "dow","date_id","Hour","timestamp"]].rename(columns={"timestamp":"Number"}).set_index(["dow","date_id","Hour"])
    
    #Number of connections per days
    Data_D__ = pd.DataFrame(data.groupby("dow").count().date_id).rename(columns={"date_id":"Number"})
    
    #Maximum gap time
    Data_gap  = data[data.Time<=np.argmax(data.set_index(["Time","dow"]).Gap_Time)[0].strftime("%Y-%m-%d %H:%M:%S")].sort_values("Time").tail(2)
    
    #Maximum distance 
    Data_max_distance_ = data[data.Time<=data.iloc[np.argmax(data.Distance_1)].Time][["Distance_1","Gap_Time_second","Localisation","Time"]].tail(2)
    
    #Maximum distance in a minimum time
    Data_max_distance_min_gap = data[data.Time<=data.iloc[np.argmin(data.Gap_Time_second/data.Distance_1)].Time][["Distance_1","Gap_Time_second","Localisation","Time"]].tail(2)
    
    print(" ")
    print("                     --- " + title + " ---")
    print("Period of analysis from {0} to {1}".format(data.Time.min(), data.Time.max()))
    print("Number of occurence over the period {0} times".format(data.shape[0]))
    print(" ")
    #print("Activity :: {0}".format(Data__))
    print("Most active day (in average over the period) : {0} - {1} times per day".format(np.argmax(data.groupby("dow").count().timestamp/data[["dow","date_id"]].groupby("dow").nunique().date_id),int(np.max(data.groupby("dow").count().timestamp/data[["dow","date_id"]].groupby("dow").nunique().date_id))))
    print("Most active day on {0} ({1}), {2} times".format(np.argmax(Data__.Number)[0],np.argmax(Data__.Number)[1],np.max(Data__.Number)))
    print("Most active per hour on {0} ({1}) between {2}h00 and {3}h00, {4} times".format(np.argmax(Data_H__.Number)[0],
                                                                                      np.argmax(Data_H__.Number)[1],
                                                                                      np.argmax(Data_H__.Number)[2],
                                                                                      np.argmax(Data_H__.Number)[2]+1,np.max(Data_H__.Number)))
    print(" ")
    print("Average time between each connection {0} (std : {1})".format(data.Gap_Time.mean(),data.Gap_Time.std()))
    print(" ")
    print("Longest time gap between connection : {2} (between {0} and {1})".format(Data_gap.Time.iloc[0],Data_gap.Time.iloc[1],str(Data_gap.Gap_Time.iloc[1])))
    print("Longest distance between connection : {0} meters (between {1} and {2} : {3} hours)".format(round(Data_max_distance_["Distance_1"].iloc[1],2),Data_max_distance_["Time"].iloc[0],Data_max_distance_["Time"].iloc[1], round(Data_max_distance_["Gap_Time_second"].iloc[1]/3600.,2)))
    print("Longest distance in a minimum time between connection : {0} meters in {1} seconds (between {2} and {3})".format(round(Data_max_distance_min_gap["Distance_1"].iloc[1],2),
                                                                                                        Data_max_distance_min_gap["Gap_Time_second"].iloc[1],
                                                                                                        Data_max_distance_min_gap["Time"].iloc[0],
                                                                                                        Data_max_distance_min_gap["Time"].iloc[1]))
    if displayed :
        display_side_by_side(Data_D__,Data__,Data_H__)
        display(data_gap)
    print("                     --- " + "---" + " ---")
    return None

## Graph functions

In [ ]:
def Plot3Dly_Users():
    import random
    colors_py = ['rgb('+str(int(random.uniform(0,255)))+','+ str(int(random.uniform(0,255)))+','+ str(int(random.uniform(0,255)))+')' for i in range(4)]
    plotly.tools.set_credentials_file(username='OAO', api_key='6UY5GWaQC2YQQzztK8Yp')
    plotly.tools.set_config_file(world_readable=True)
    mapbox_access_token = "pk.eyJ1Ijoib2FvIiwiYSI6ImNqbXQzZTVlaDFtdHcza3BuN2xxZmh0NG0ifQ.NrLjY1_AoNExI8DRWNHMcg"
    data = [
        go.Scattermapbox(
            lat=[i+round(random.uniform(0,10),5) for i in data.sample(100).latitude],
            lon=[i+round(random.uniform(0,10),5) for i in data.sample(100).longitude],
            text = ["Connection n°{0} : {1}".format(i,str(j)) for i,j in enumerate(data.sample(100).index)],
            mode='markers',
            #customdata = z_,
            marker=dict(
                 size=9,
                color=colors_py[i],
                opacity=0.7
            ),
            hoverinfo='text'
        ) for i,data in enumerate(List_Data)]

    layout = go.Layout(
        title='Activity of users',
        autosize=True,
        hovermode='closest',
        showlegend=True,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat=List_Data[0].latitude.mean(),
                lon=List_Data[0].longitude.mean()
            ),
            pitch=0,
            zoom=3,
            style='light'
        ),
    )
    fig = dict(data=data, layout=layout)
    return fig

In [ ]:
def Plot3Dly_User1_abnormal():
    colors_py = ['rgb('+str(int(random.uniform(0,255)))+','+ str(int(random.uniform(0,255)))+','+ str(int(random.uniform(0,255)))+')']
    plotly.tools.set_credentials_file(username='', api_key='')
    plotly.tools.set_config_file(world_readable=True)
    site_lat = Model_users_Distance[0].data[(Model_users_Distance[0].data.index>="2015-03-18 05:51:18")&(Model_users_Distance[0].data.index<="2015-03-18 06:02:52")].latitude.values
    site_lon = Model_users_Distance[0].data[(Model_users_Distance[0].data.index>="2015-03-18 05:51:18")&(Model_users_Distance[0].data.index<="2015-03-18 06:02:52")].longitude.values
    Location_text = ["Connection n°{0} : {1}".format(i,data) for i, data in enumerate(Model_users_Distance[0].data[(Model_users_Distance[0].data.index>="2015-03-18 05:51:18")&(Model_users_Distance[0].data.index<="2015-03-18 06:02:52")].Loc__.values)]
    z_ = Model_users_Distance[0].data[(Model_users_Distance[0].data.index>="2015-03-18 05:51:18")&(Model_users_Distance[0].data.index<="2015-03-18 06:02:52")].Rank_location.values
    mapbox_access_token = "pk...."
    data = [
        go.Scattermapbox(
            lat=site_lat,
            lon=site_lon,
            text =Location_text,
            mode='markers',
            marker=dict(
                 size= np.log(z_*(z_+100))*2,
                color=colors_py[0],
                opacity=0.5
            ),
            hoverinfo='text'
        )]

    layout = go.Layout(
        title='Abnormal connections of user 1',
        autosize=True,
        hovermode='closest',
        showlegend=False,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat= site_lat.mean(),
                lon= site_lon.mean()
            ),
            pitch=0,
            zoom=3,
            style='light'
        ),
    )
    fig = dict(data=data, layout=layout)
    return fig